<a href="https://colab.research.google.com/github/tanmaychikhale/MachineLearning/blob/main/Practical_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    confusion_matrix,
    classification_report
)

In [12]:
file_path = '/content/WA_Fn-UseC_-Telco-Customer-Churn.csv'
churn_data = pd.read_csv(file_path)

In [13]:
# Step 3: Preprocess the Data
# Convert TotalCharges to numeric and handle missing values
churn_data['TotalCharges'] = pd.to_numeric(churn_data['TotalCharges'], errors='coerce')
churn_data.dropna(subset=['TotalCharges'], inplace=True)

In [14]:
# Encode target variable 'Churn' (Yes -> 1, No -> 0)
churn_data['Churn'] = churn_data['Churn'].map({'Yes': 1, 'No': 0})

In [15]:
# Drop 'customerID' (not useful for modeling)
churn_data = churn_data.drop(columns=['customerID'])

In [16]:
# Encode categorical variables using LabelEncoder
label_encoders = {}
for column in churn_data.select_dtypes(include='object').columns:
    le = LabelEncoder()
    churn_data[column] = le.fit_transform(churn_data[column])
    label_encoders[column] = le

In [17]:
# Step 4: Split Data into Training and Testing Sets
X = churn_data.drop(columns=['Churn'])
y = churn_data['Churn']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


In [18]:
# Step 5: Train a Random Forest Classifier
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [19]:
# Step 6: Evaluate the Model
y_pred = clf.predict(X_test)
y_prob = clf.predict_proba(X_test)[:, 1]

In [20]:
# Step 7: Calculate and Display Evaluation Metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_prob)
conf_matrix = confusion_matrix(y_test, y_pred)

In [21]:
print("Evaluation Metrics:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")
print(f"ROC-AUC: {roc_auc:.4f}")
print("\nConfusion Matrix:")
print(conf_matrix)

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

Evaluation Metrics:
Accuracy: 0.7832
Precision: 0.6177
Recall: 0.4840
F1-Score: 0.5427
ROC-AUC: 0.8113

Confusion Matrix:
[[921 112]
 [193 181]]

Classification Report:
              precision    recall  f1-score   support

           0       0.83      0.89      0.86      1033
           1       0.62      0.48      0.54       374

    accuracy                           0.78      1407
   macro avg       0.72      0.69      0.70      1407
weighted avg       0.77      0.78      0.77      1407

